In [5]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.memory import ConversationBufferMemory
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain import PromptTemplate
import faiss
import streamlit as st
import os


ImportError: cannot import name 'field_validator' from 'pydantic' (c:\Users\USER\anaconda3\Lib\site-packages\pydantic\__init__.cp311-win_amd64.pyd)

In [6]:
doc_loader = DirectoryLoader(path="data/", glob="*.pdf", loader_cls=PyPDFLoader)
documents = doc_loader.load()

In [2]:
os.chdir("../")

In [8]:
# split the documents into chunks 

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, 
                                               chunk_overlap=20)
text_chunks = text_splitter.split_documents(documents)

In [9]:
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", 
                                   model_kwargs={"device": "cpu"})

C:\Users\eigen\AppData\Local\Temp\ipykernel_10164\1360172283.py:1: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2", model_kwargs={"device": "cpu"})
c:\Users\eigen\anaconda3\envs\tf-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [10]:
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)
vector_store.save_local("vector_store/faiss_db")

In [13]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

C:\Users\eigen\AppData\Local\Temp\ipykernel_10164\3353824168.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


In [12]:
llm = OpenAI(model_name="gpt-4")

In [ ]:
chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    chain_type="stuff", 
    retriever=vector_store(search_kwargs={"k":3}), 
    memory=memory
)

In [ ]:
query = "In which century did the French psychiatrists, Ambrose Tardieu and Paul-Max Simon both \
        published studies on the similar characteristics of and symbolism in \
        the artwork of the mentally ill."
response = chain.run({"question": query})
print(response)
